In [1]:
import requests
import os
import json
import pandas as pd
import datetime

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
# bearer_token = os.environ.get("BEARER_TOKEN")
bearer_token ='AAAAAAAAAAAAAAAAAAAAACqHUgEAAAAAcghJSkKBmFmw36ZiRxrqVdyomIk%3DZlDqMjNA0ka4btIuhDjiHUyRGoCHDzKBWvNGcqYXpLJ5kuhUAp'

In [2]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [3]:
class Scrapper:
    def __init__(self, keyword):
        self.keyword = keyword
        # Creates the dataframe for the keyword
        self.dataframe = pd.DataFrame()
    
    def file_name(self):
        return f'data/tweets_{self.keyword}.csv'
    
    def get_tweets(self, from_date):
        to_date = from_date + datetime.timedelta(days=1)
        search_url = "https://api.twitter.com/2/tweets/search/recent"
        query_params = {
            'query': self.keyword, 
            'start_time': from_date.isoformat("T") + "Z", 
            'end_time': to_date.isoformat("T") + "Z",
            'tweet.fields': 'author_id,created_at,geo', 
            'max_results': 100}
        print(query_params)
        # Queries Twitter to get the keyword related tweets
        json_response = connect_to_endpoint(search_url, query_params)
        # Convert the response to a dataframe
        df = pd.DataFrame(json_response['data']) 
        df['Keyword'] = self.keyword
        # Appends the results to the planned day dataframe
        self.dataframe = self.dataframe.append(df, ignore_index=True)

    def check_file_existence(self):
        return os.path.exists(self.file_name())
        
    def save(self):
        print(f'Saving {self.file_name()}')
        self.dataframe.to_csv(self.file_name())

In [4]:
dates = [
    datetime.datetime(2021,10,13,13),
    datetime.datetime(2021,10,14),
    datetime.datetime(2021,10,15),
    datetime.datetime(2021,10,16),
    datetime.datetime(2021,10,17),
    datetime.datetime(2021,10,18),
    datetime.datetime(2021,10,19),
]
keywords = ['altcoin', 'bitcoin', 'coindesk', 'cryptocurrency', 'gold', 'APPL', 'GOOG', 'YHOO']

In [10]:
# Executes the planned queries for each keyword
for keyword in keywords:
    scrapper = Scrapper(keyword)
    if scrapper.check_file_existence():
        # If the file already exists ommits the query to twitter API
        print(f'File {scrapper.file_name()} already exists.. omiting query')
        continue
    # Gets tweet for each date
    for date in dates:
        scrapper.get_tweets(date)
    # Saves the file
    scrapper.save()       
        

NameError: name 'item' is not defined

In [7]:
# Reads the tweets json files and consolidates them in a single dataframe
complete_df = pd.DataFrame() 
for keyword in keywords:
    scrapper = Scrapper(keyword)
    df = pd.read_csv(scrapper.file_name())
    complete_df = complete_df.append(df, ignore_index=True)

In [8]:
complete_df.sample(10)

,Unnamed: 0,created_at,author_id,text,id,Keyword,geo
88,88,2021-10-14T12:54:59.000Z,971433276761141249,#Ethereum fiyatı en üst sırayı kırmaya ve 4.08...,1448633431768829953,altcoin,NaN
4076,610,2021-10-19T23:03:52.000Z,976878775471345670,RT @alison_bowden: Incredible job opportunity ...,1450598603861344263,APPL,NaN
3084,310,2021-10-16T23:59:48.000Z,2535222656,RT @FangasmSPN: .@jarpad @JensenAckles hug to ...,1449525514436292609,gold,NaN
1735,354,2021-10-16T23:10:11.000Z,1365392039760728064,"RT @We_Are_Many__: Hold your Strength, WE are ...",1449513028991348741,coindesk,NaN
7,7,2021-10-14T12:59:53.000Z,1447208551395430400,@gate_io @elonmusk @gate_io Buff Doge Coin to...,1448634667679162375,altcoin,NaN
4166,2,2021-10-14T12:56:57.000Z,2327404376,"Oleg Boyko, via his Finstar, was (at one time)...",1448633926675800067,GOOG,NaN
1324,637,2021-10-19T23:59:53.000Z,752656592928202752,@davidgokhshtein Guys check bitcoin ATH! 🚀,1450612699809845256,bitcoin,NaN
2240,165,2021-10-14T23:59:06.000Z,1407935031096475650,@ElpisBattle Good project is very good and thi...,1448800562468716544,cryptocurrency,NaN
3522,56,2021-10-14T10:28:28.000Z,808319497849421824,@_appl_pie эх бубсы( \nхочу Итто(((,1448596562473635846,APPL,NaN
2171,96,2021-10-14T12:59:21.000Z,967060789143265280,I think this project will be successful as tra...,1448634533956317185,cryptocurrency,NaN
